# 노트북 소개
해당 노트북은 Interactive Porto Insights - A Plot.ly Tutorial(https://www.kaggle.com/arthurtok/interactive-porto-insights-a-plot-ly-tutorial)의 한글 번역본입니다!

필사를 하며 개인적으로 느꼈던 점과 활용한 방법들도 기재가 되어있습니다.
오역과 의역이 다수 있으니 잘못된 점 알려주시면 수정하도록 하겠습니다.

# Introduction
브라질에서 세번째로 큰 보험 회사인 Porto Seguro에 의해 주최된 이 경쟁은 운전자가 내년에 보험 청구를 할 가능성에 대해 예측하는 작업을 진행합니다.

이 노트북은 파이썬의 시각화 도구인 Plot.ly를 활용하여 상호관계적 차트를 만들고 데이터를 분석하는 하는데 목적을 두고 있습니다. Plot.ly는 통계적 시각화에 특화된 라이브러리로 파이썬뿐만 아니라 다른 프로그래밍 언어에서도 사용하고 있습니다.

우리는 Plot.ly를 통하여 편리하게 다양한 그래프들을 만들어볼 수 있습니다.
1. Simple Horizontal bar plot : 단순 수평 막대그래프 
2. Correlation Heatmap plot : 상관관계 히트맵
3. Scatter Plot : 산점도
4. Vertical Bar plot : 수직 막대그래프트
5. 3D Scatter Plot : 3D 산점도

이 노트북의 테마는 다음과 같이 정리 가능합니다:
1. 데이터 품질 점검 - 결측값을 시각화하고 평가합니다.
2. Feature 점검과 필터링 - 상관관계와 상호의존정보를 활용합니다. Binary, Categorical 그리고 다른 변수들을 점검합니다.
3. 학습한 모델을 활용한 Feature 중요도 랭킹 산정 - Random Forest와 Gradient Boosting 을 활용하여 모델을 만들고 학습 과정에 기반하여 Feature들의 중요도 순위를 산정합니다.

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import plotly.offline as py
import plotly.graph_objs as go
import plotly.tools as tls
import warnings
warnings.filterwarnings('ignore')
from collections import Counter
from sklearn.feature_selection import mutual_info_classif

plotly.offline : 만들어진 그래프를 시각화합니다.

plotly.graph_objs : 그래프를 선택하고 데이터와 레이아웃을 설정합니다.

plotly.tools : 이번 노트북에서 사용하지 않습니다.

In [ ]:
train = pd.read_csv('../input/porto-seguro-safe-driver-prediction/train.csv')
train.head()

In [ ]:
# train 데이터의 행과 열의 갯수를 확인합니다
rows = train.shape[0]
columns = train.shape[1]

print('Train dataset contains {0} rows and {1} columns.'.format(rows, columns))

### Data Quality checks
결측값을 찾아봅니다.

In [ ]:
# 1. 데이터 퀄리티 체크

# 방법 1 any().any() 활용
print(train.isnull().any().any())

# 방법 2 sum().sum() 활용
print(train.isnull().sum().sum())

원본 작성자는 any().any()를 활용했지만, 개인적으로는 sum().sum()이 더 낫다고 생각해서 두 개 다 넣어놨습니다.

결측값이 전혀 없는 듯하지만 데이터 설명에 따르면 결측값은 -1로 처리가되어있음을 알 수 있습니다. 따라서 -1 값을 NaN 값으로 치환해보도록 하겠습니다.

In [ ]:
train_copy = train
train_copy = train_copy.replace(-1, np.NaN)

count_null = train_copy.isnull().sum().sum()
print('치환된 Null값의 갯수는 {}개입니다'.format(count_null))

치환된 Null값의 개수를 확인했습니다.

이제 Missingno 패키지를 활용하여 결측값을 확인해보도록 하겠습니다.

In [ ]:
# missingno 를 활용하여 Null값 확인하기
import missingno as msno
msno.matrix(df=train_copy.iloc[:, 2:39],
           figsize=(20,14), color=(0.42, 0.1, 0.05))

와인색 부분은 데이터가 온전히 보존된 부분이고 흰색 부분은 데이터가 유실된 부분을 나타냅니다. 

이를 통해 우리는 59개의 Feature들 중 7개의 Feature에 결측값이 있음을 확인할 수 있습니다. 하지만 엄밀히 따지면 총 13개의 Feature에 결측값이 있습니다. Missingno 매트릭스는 대략 40개의 Feature들에 맞추어져 있기 때문에 제외된 Feature들에서 결측값을 지닌 남은 5개의 Feature들이 발견됐을 것입니다.

우리가 발견한 7개의 칼럼은 다음과 같습니다.

**ps_ind_05_cat | ps_reg_03 | ps_car_03_cat | ps_car_05_cat | ps_car_07_cat | ps_car_09_cat | ps_car_14**

많은 결측값을 가진 칼럼은 '_cat'이라는 접미사를 가지고 있습니다. 원본 노트북에는 없지만 결측값의 비율을 한 번 확인해봅시다.

In [ ]:
# Null값의 퍼센트 비율을 확인
for col in train_copy :
    if train_copy[col].isnull().sum() > 0 :
        print('{} contains {} Null values ({:.2f}%)'.format(col, train_copy[col].isnull().sum(),
                                                                               train_copy[col].isnull().sum()/len(train_copy[col])))

앞서 언급했듯이 13개의 칼럼들이 존재함을 알 수 있습니다.

**Target 변수 검사**

표준화된 검사 과정엔 Target Variable에 대한 검사가 수행됩니다. 이번 데이터의 경우에는 이미 Target Variable이 'target'이라는 이름으로 지정되어 있습니다. 이 Target Variable은 class/label/correct라는 별칭으로 불리기도 하며 이를 바탕으로 새로운 데이터 값이 주어졌을 때 미지의 값을 예측하게 됩니다.

In [ ]:
# 타겟 변수를 확인

data = [go.Bar(x=train['target'].value_counts().index,
               y=train['target'].value_counts().values,
               text='Distribution of target variable'
              )]

layout = go.Layout(title='Target variable distribution')

fig = go.Figure(data=data, layout=layout)

py.iplot(fig)

matplotlib과 seaborn이 아닌 **Plot.ly**로 그래프를 그려봤습니다.

과정은 다음과 같이 진행됐습니다.

1. go.Bar를 활용하여 x축과 y축 데이터를 지정해줍니다. text는 그래프에 마우스를 올렸을 때 나타나는 텍스트를 의미합니다.
2. go.Layout을 활용하여 그래프의 제목을 지정해줍니다.
3. go.Figure를 활용하여 fig에 그래프를 할당해줍니다.
4. py.iplot을 활용하여 그래프를 시각화해줍니다.

여기서 go는 plotly.graphic_objs, iplot은 plotly.offline이었습니다. 기억해두도록 합시다.

시각화된 데이터를 보니 레이블값이 매우 불균형함을 알 수 있습니다.

**Datatype check**

이 과정을 통해 훈련 데이터가 어떤 데이터들로 구성되어 있는지 확인해봅니다. 정수인지, 문자인지, 실수인지 확인함으로써 데이터에 대해 더 나은 전반적인 이해를 얻을 수 있습니다. Collections 모듈의 Counter를 활용하여 확인해보도록 합니다.

In [ ]:
# 훈련 데이터의 타입 확인 (방법 1)
train.dtypes.value_counts().to_frame()

개인적으로 저는 굳이 Collections의 Counter를 사용할 이유를 못느껴 value_counts를 frame으로 만들어 확인했습니다.

In [ ]:
# 훈련 데이터의 타입 확인 (방법 2)
Counter(train.dtypes.values)

59개로 이루어진 칼럼들은 39개의 int, 20개의 float으로 구성되어 있습니다.

각 데이터들은 '_bin', '_cat', '_reg'라는 접미사를 가지고 있습니다. 각각의 접미사들은 뜻을 가지고 있는데, '_bin'은 binary, '_cat'은 categorical, '_reg'는 Continious 이거나 Ordianl 데이터를 말합니다.

이제 단순히 float values(Continious Feature)와 integer values(Binary, Categorical, Ordinal)으로 데이터를 나눠보도록 합시다.

In [ ]:
# float과 int를 select_dtypes를 활용하여 구분
train_float = train.select_dtypes(include=['float64'])
train_int = train.select_dtypes(include=['int64'])

### Correlation plots
변수들의 선형 상관관계 그래프를 확인해봄으로써 인사이트를 얻어보도록 하겠습니다. 일단 seaborn 라이브러리를 활용하여 히트맵을 그려보도록 합시다. 편리하게 판다스의 corr()을 사용하여 피어슨 상관계수를 계산해준 뒤, seaborn의 heatmap을 사용해보도록 합시다.

**float features들의 상관관계**

In [ ]:
# seaborn을 활용하여 float 데이터의 히트맵 생성
colormap = plt.cm.magma
plt.figure(figsize=(16,12))
plt.title('Pearson correlation of continuous features', y=1.05, size=15)
sns.heatmap(train_float.corr(),linewidths=0.1,vmax=1.0, square=True, 
            cmap=colormap, linecolor='white', annot=True)

히트맵을 통해 대다수의 Featurese들이 0이거나 다른 변수들과 상관관계가 거의 없음을 확인 가능합니다. 이것은 추가로 할 조사에 흥미로운 관찰 결과를 말해줍니다. 지금 당장에는, 양의 상관관계를 보이는 변수들은 다음과 같습니다.

**(ps_reg_01, ps_reg_03)**

**(ps_reg_02, ps_reg_03)**

**(ps_car_12, ps_car_13)**

**(ps_car_13, ps_car_15)** 

**Integer Features들의 상관관계**

이번에는 seaborn 대신 plotly를 사용해보도록 하겠습니다.
간단하게 'go.Heatmap'을 사용하면 됩니다. x축과 y축에는 각 칼럼의 이름들을 기입해주고 z축에는 상관관계 값을 기입해주도록 합시다.

In [ ]:
data = [
    go.Heatmap(
        z=train_int.corr().values,
        x=train_int.columns.values,
        y=train_int.columns.values,
        colorscale='Viridis',
        reversescale=False,
        opacity=1.0)
]

layout = go.Layout(
        title='Peason corr of Integer-type Features',
        xaxis=dict(ticks='', nticks=36),
        yaxis=dict(ticks=''),
        width=900, height=700)

fig = go.Figure(data=data, layout=layout)

py.iplot(fig)

비슷하게, 대다수의 칼럼들이 상관관계를 가지고 있지 않거나 0임을 확인 가능합니다. 이 정보는 만약 우리가 주성분 분석(PCA)과 같이 차원 축소를 진행하는 경우에 매우 유용한 정보가 될 수 있습니다. 

우리에게 흥미로운 칼럼은 다음과 같습니다


**음의 상관관계를 가지는 Features: ps_ind_06_bin, ps_ind_07_bin, ps_ind_08_bin, ps_ind_09_bin**

또 다른 흥미로운 측면은 우리가 결측값 분석에서 많은 결측값을 지녔던 ps_car_03_cat과 ps_car_05_cat에서 찾아볼 수 있습니다. 우리는 이 둘이 강력한 양의 상관을 가지고 있어도 놀라울 일이 아니지만 데이터에 대한 근본적인 진실을 제대로 반영하지 못하고 있을 수 있습니다. 

### Mutual Information plots
상호의존정보는 Target과 변수들 간의 상호 정보를 검사할 수 있는 유용한 도구입니다. 분류 문제에서, 우리는 sklearn의 mutual_info_classif 메소드로 손쉽게 이를 측정할 수 있습니다. 0일 경우 랜덤한 변수이 서로 독립적이라는 것을 의미하며, 값이 높아질수록 약간의 독립성을 나타냅니다. 이 기능을 통해 Feature 내부에 target에 대한 정보가 얼마나 담겨있는지 알 수 있습니다.

In [ ]:
#상호의존정보 확인
mf = mutual_info_classif(train_float.values,
                        train['target'].values,
                        n_neighbors=3,
                        random_state=17)

print(mf)

###  Binary features inspection
다음으로는 0과 1, 두 개의 값만을 가지고 있는 binary columns들에 대한 검사를 시작합니다. 모든 binary 칼럼을 호출하여 0과 1의 비율을 그래프로 나타내보도록 합시다.

In [ ]:
bin_col = []
for col in train.columns :
    if '_bin' in col :
        bin_col.append(col)
        
zero_list = []
one_list = []

for col in bin_col :
    zero_list.append((train[col]==0).sum())
    one_list.append((train[col]==1).sum())

In [ ]:
trace1 = go.Bar(
        x=bin_col,
        y=zero_list,
        name='Zero count')

trace2 = go.Bar(
        x=bin_col,
        y=one_list,
        name='One Count')

data=[trace1, trace2]

layout = go.Layout(
        barmode='stack',
        title='Count of 1 and 0 in binary variables')

fig = go.Figure(data=data, layout=layout)

py.iplot(fig)

*ps_ind_10_bin, ps_ind_11_bin, ps_ind_12_bin, ps_ind_13_bin* 이 네 개의 칼럼은 거의 완벽히 0에게 지배당했습니다. 이를 통해 target에 대한 정보가 거의 담기지 않았음을 유추할 수 있을 것입니다.

### 랜덤 포레스트를 활용한 Featrue importance 확인


이제 랜덤 포레스트 분류기로 훈련 데이터를 fit 시키고, 모델이 훈련을 마친 후 특징의 순위를 살펴보는 랜덤 포레스트 모델을 구현하도록 합니다. 이것은 유용한 Feature importance를 얻는 데 많은 하이퍼 파라미터 튜닝이 필요하지 않고 목표 불균형을 파악하는 데도 매우 강력한 앙상블 모델을 사용하는 빠른 방법입니다.

In [ ]:
# Random Forest를 활용한 Feature 중요도 확인
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=150,
                           max_depth=8,
                            min_samples_leaf=4,
                            max_features=0.2,
                            n_jobs=-1,
                            random_state=0)

X_train = train.drop(['target', 'id'], axis=1)
y_train = train['target']

rf.fit(X_train, y_train)

features = train.drop(['target' , 'id'], axis=1).columns.values

print('Training Done')

**Plot.ly Scatter Plot**

훈련된 랜덤 포레스트 모델은 "feature_importances_"로 값을 불러올 수 있습니다. 그렇다면 plotly의 Scatter plot으로 이를 확인해보도록 합시다. 한가지 주의해야 할 점은 scatter plot의 Marker 속성입니다. 이를 통해 사이즈, 색상, 스케일 등을 조정할 수 있습니다.

In [ ]:
trace = go.Scatter(
    y = rf.feature_importances_,
    x = features,
    mode='markers',
    marker=dict(
        sizemode = 'diameter',
        sizeref = 1,
        size = 13,
        #size= rf.feature_importances_,
        #color = np.random.randn(500), #set color equal to a variable
        color = rf.feature_importances_,
        colorscale='Portland',
        showscale=True
    ),
    text = features
)
data = [trace]

layout= go.Layout(
    autosize= True,
    title= 'Random Forest Feature Importance',
    hovermode= 'closest',
     xaxis= dict(
         ticklen= 5,
         showgrid=False,
        zeroline=False,
        showline=False
     ),
    yaxis=dict(
        title= 'Feature Importance',
        showgrid=False,
        zeroline=False,
        ticklen= 5,
        gridwidth= 2
    ),
    showlegend= False
)
fig = go.Figure(data=data, layout=layout)
py.iplot(fig,filename='scatter2010')

막대 그래프 형태로도 살펴보도록 합시다.

In [ ]:
x, y = (list(x) for x in zip(*sorted(zip(rf.feature_importances_, features), 
                                                            reverse = False)))
trace2 = go.Bar(
    x=x ,
    y=y,
    marker=dict(
        color=x,
        colorscale = 'Viridis',
        reversescale = True
    ),
    name='Random Forest Feature importance',
    orientation='h',
)

layout = dict(
    title='Barplot of Feature importances',
     width = 900, height = 2000,
    yaxis=dict(
        showgrid=False,
        showline=False,
        showticklabels=True,
#         domain=[0, 0.85],
    ))

fig1 = go.Figure(data=[trace2])
fig1['layout'].update(layout)
py.iplot(fig1, filename='plots')

### Gradient Boosting을 통한 Featrue importance 확인

다른 모델을 활용하여 Feature importances를 얻어보도록 합시다.

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
gb = GradientBoostingClassifier(n_estimators=100,
                                max_depth=3,
                               min_samples_leaf=4,
                               max_features=0.2,
                               random_state=0)

gb.fit(X_train, y_train)

print('Training Done')

In [ ]:
trace = go.Scatter(
    y = gb.feature_importances_,
    x = features,
    mode='markers',
    marker=dict(
        sizemode = 'diameter',
        sizeref = 1,
        size = 13,
        color = gb.feature_importances_,
        colorscale='Portland',
        showscale=True
    ),
    text = features
)
data = [trace]

layout= go.Layout(
    autosize= True,
    title= 'Gradient Boosting Machine Feature Importance',
    hovermode= 'closest',
     xaxis= dict(
         ticklen= 5,
         showgrid=False,
        zeroline=False,
        showline=False
     ),
    yaxis=dict(
        title= 'Feature Importance',
        showgrid=False,
        zeroline=False,
        ticklen= 5,
        gridwidth= 2
    ),
    showlegend= False
)
fig = go.Figure(data=data, layout=layout)
py.iplot(fig,filename='scatter2010')

In [ ]:
x, y = (list(x) for x in zip(*sorted(zip(gb.feature_importances_, features), 
                                                            reverse = False)))
trace2 = go.Bar(
    x=x ,
    y=y,
    marker=dict(
        color=x,
        colorscale = 'Viridis',
        reversescale = True
    ),
    name='Random Forest Feature importance',
    orientation='h',
)

layout = dict(
    title='Barplot of Feature importances',
     width = 900, height = 2000,
    yaxis=dict(
        showgrid=False,
        showline=False,
        showticklabels=True,
#         domain=[0, 0.85],
    ))

fig1 = go.Figure(data=[trace2])
fig1['layout'].update(layout)
py.iplot(fig1, filename='plots')

흥미롭게도 두 모델 모두에게서 **ps_car_13**이 제일 중요한 칼럼임을 확인할 수 있었습니다.